In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from supabase import create_client, Client
supabase_url = os.environ.get("SUPABASE_URL")
supabase_api_key = os.environ.get("SUPBASE_KEY")

supabase: Client = create_client(supabase_url, supabase_api_key)

In [2]:
from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [5]:
def model_research(date):
    result = supabase.table('curation_selected_items')\
        .select('event_id, output,sources, news_date','topic')\
    .eq('news_date', date)\
    .eq('topic', 'Model announcements/enhancements')\
        .order('created_at', desc=False)\
        .execute()
    
    return result.data

In [9]:
funding_deep_dive = funding_research('2026-02-21')

In [14]:
def openai_research_model(research_input):
    for i in research_input:
        response = client.responses.create(
            model = "gpt-5-nano",
            tools = [{
                "type": "web_search"
            }],
            include = ["web_search_call.action.sources"],
            input = f"""
           You are a research analyst for Krux.

          TASK:
          Create a fact-checked research brief for ONE model announcement/enhancement.
        
          GOAL:
          Explain what was announced, what changed, and why it matters in simple language for working teams.
        
          OUTPUT FORMAT:
          - 8 to 10 bullet points max.
          - Each bullet must end with inline citations:
            [Source: <publisher>, <url>]
          - No text before or after bullets.
        
          MANDATORY STRUCTURE:
          1) WHAT IS NEW (first bullet, one sentence)
             - Exact announcement today (new model/version/feature rollout).
          2) WHAT CHANGED vs BEFORE
             - New capabilities, limits, speed/cost/context/tool-use changes.
             - If comparison is not explicitly available, write:
               "Not explicitly benchmarked in cited sources."
          3) AVAILABILITY
             - Who gets access (free/pro/enterprise/API), regions, rollout timeline.
          4) PRACTICAL USE CASES
             - 2 concrete workflows this enables, explained in plain language.
          5) TEAM IMPACT (CONDITIONAL)
             - Always include ONE unified "team impact" bullet relevant across roles.
             - If the launch clearly impacts one role more (e.g., designers/developers/marketers/product managers),
               add ONLY ONE extra role-specific bullet.
             - Do NOT create separate bullets for every persona.
          6) RISKS / LIMITS
             - Known constraints, safety caveats, reliability limitations.
          7) WHY THIS MATTERS NOW
             - One factual bullet on relevance or impact.
             - The impact on a specific industry, stock, sector etc
        
          STRICT RULES:
          - Keep language simple and non-jargony.
          - No hype, no opinions, no predictions.
          - Every claim must be source-backed.
          - Prefer primary/high-signal sources (official docs/releases + major outlets).
          - If sources conflict, include both and label as conflicting.
          - No repeated facts across bullets.
        
          QUALITY CHECK BEFORE FINALIZING:
          - First bullet clearly states the new announcement.
          - At least one impact bullet is practical and actionable for teams.
          - All bullets contain citations.
          - Output is concise and non-redundant.
          
          Event to research:
              
              Event: {i['output']}
              
              Sources: {i['sources']}
            """,
        )

        output = response.output_text
        print(output)

        final_dictionary = {
            'event_id': i['event_id'],
            'news_date': i['news_date'],
            'output': output,
            'model_provider': 'openai',
            'topic': i['topic']
        }
        print(final_dictionary)

        save_research(final_dictionary)

        print("✅ Done")

In [15]:
funding_deep_dive[0:1]

[{'event_id': '1300_2026-02-21',
  'output': 'Anthropic launched Claude Code Security, a security-focused product built using its Opus 4.6 model family. The tool can autonomously review large codebases, surface and prioritize high-severity vulnerabilities, double-check its findings, and suggest fixes; it is being released as a limited research preview for Enterprise/Team customers and open-source maintainers, with safeguards and cautious rollout due to dual-use risks.',
  'sources': [{'url': 'https://fortune.com/2026/02/20/exclusive-anthropic-rolls-out-ai-tool-that-can-hunt-software-bugs-on-its-own-including-the-most-dangerous-ones-humans-miss/',
    'name': 'Fortune'},
   {'url': 'https://www.pcmag.com/news/anthropic-rolls-out-autonomous-vulnerability-hunting-ai-tool-for-claude',
    'name': 'PCMag'}],
  'news_date': '2026-02-21',
  'topic': 'Model announcements/enhancements'}]

In [ ]:
openai_research_model(funding_deep_dive[0:1])